In [ ]:
%matplotlib widget

import numpy as np
import cupy as cp
import cupyx.scipy.fft as cpfft
import scipy.fft as spfft
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib_scalebar.scalebar import ScaleBar
import time
from pathlib import Path

from fastssb import data4d as d4
from fastssb.data4d import Metadata4D, load_sparse_4dcam_data, calculate_pacbed_from_sparse
from fastssb.optics import wavelength, Q_freq_to_d_spacing, d_spacing_to_Q_freq, repr_d_spacings, get_qx_qy_1D, disk_overlap_function, single_sideband_reconstruction
from fastssb.plotting import fft_doverlap_figure, cx_to_hsv_img, manual_aberration_ui

import tifffile

plt.rcParams['figure.dpi'] = 100
size12 = (12.8,4.8)
b_GB = 1073741824
b_MB = 1048576

In [ ]:
# load my data set

data_dir = '/global/cfs/projectdirs/ncemhub/distiller'
base_path = Path(data_dir)

scan_number, scan_id = 57, 3323
date = Path('2022.08.30')
sparse_path = base_path / Path('counted') / Path(date)

adfpath = base_path / Path('dm4') / Path(date)
results_path = base_path / Path('results')

if not results_path.exists():
    results_path.mkdir()

filename4d = sparse_path / f'data_scan{scan_number}_id{scan_id}_electrons.h5'
# filename4d = sparse_path / f'data_scan{scan_num}_th{th}_electrons.h5'
# filename4d = sparse_path / f'data_scan{scan_num}_electrons.h5'
filenameadf = adfpath / f'scan{scan_number}.dm4'

print('Loading data...')
frames, scan_dimensions, frame_dimensions, max_frame_size, tot_electrons, tot_image = load_sparse_4dcam_data(filename4d) # load entire data set
# frames, scan_dimensions, frame_dimensions, max_frame_size, tot_electrons, tot_image = load_sparse_4dcam_data(filename4d, 256, (384, 384))  # load a subset of data set

meta = Metadata4D.from_dm4_file(filenameadf)  # automatically gets beam energy and scan step
meta.alpha_rad = 30.0e-3 # I also have used 17.1e-3
meta.rotation_deg = -93.1 # TEAM 0.5 at 80 kV with -30.0 deg scan rotation in TIA

#####
dwell_time = 1/87e3
detector_to_real_fluence_80kv = 1 

fluence = d4.get_fluence(tot_electrons, scan_dimensions, meta.dr[0]) * detector_to_real_fluence_80kv
flux = d4.get_flux(tot_electrons, scan_dimensions, meta.dr[0], dwell_time) * detector_to_real_fluence_80kv

print('====== Data Info =======')
print(f"E               = {meta.E_ev/1e3} keV")
print(f"λ               = {meta.wavelength * 1e2:2.2} pm")
print(f"dR              = {meta.dr} Å")  # real space pixel size
print(f"scan       size = {scan_dimensions}")
print(f"detector   size = {frame_dimensions}")
print(f"scan       FOV  = {scan_dimensions*meta.dr/10} nm")
print(f"fluence         ~ {fluence} e/Å^2")
print(f"flux            ~ {flux} e/Å^2/s")
######

pacbed = calculate_pacbed_from_sparse(frames, frame_dimensions, tot_electrons)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=size12)
im1 = ax1.imshow(pacbed)
cb1 = fig.colorbar(im1, ax=ax1)

im2 = ax2.imshow(tot_image)
cb2 = fig.colorbar(im2, ax=ax2)
im2.set_clim(np.percentile(tot_image, 1), np.percentile(tot_image, 99))
plt.show()

In [ ]:
# Get BF disk radius and center
# The plot should look like a circle. The green dot should be at the center and
# the red dots around the BF disk. If not, try adjusting the threshold?
radius_original, center = d4.locate_BF_disk(pacbed, threshold=0.1)

In [ ]:
# Decide how much to bin the CBED patterns
# The G array needs to fit in GPU memory; with a larger bin factor, the G array will be smaller.
target_frame_size = 25 #25, 32
bin_factor = frame_dimensions[0] // target_frame_size

alpha_max_factor = 1.03
radius_max = radius_original * alpha_max_factor
radius_data_int = int(np.ceil(radius_original/bin_factor) * bin_factor)
radius_max_bin_int = int(np.ceil(radius_max/bin_factor))
radius_max_int = int(radius_max_bin_int * bin_factor)
radius = radius_original/bin_factor

frame_size = 2*radius_max_bin_int + 1

# (ssb_size, bin_factor, ssb_size[0]*bin_factor, frame_dimensions)
print(f'Original frame dimensions: {frame_dimensions}')
print(f'Bin factor: {bin_factor}')
print(f'Final frame size: {frame_size} px')
print(f'Frame radius: {radius_max_bin_int} px, radius of BF disk: {radius:.2f} px')
G_nbytes_expected = 8 * frame_size**2 * scan_dimensions[0] * scan_dimensions[1]
print(f'G array expected to require {G_nbytes_expected/b_GB:.2f} GB on the GPU')

# We are going to crop and bin the diffraction pattern to a square array with dimensions frame_size x frame_size.
# Use convention that frame_size is odd, with frequency = 0 located at frame_size // 2.
# The corresponding spatial frequencies will later be determined by fftfreq.

cmap = mpl.cm.get_cmap('viridis').copy()
cmap.set_under(color='black')
pacbed_binned, _ = d4.crop_bin_single_CBED(frame_size, bin_factor, center, frame_dimensions, pacbed)
min_nonzero = np.amin(pacbed_binned[pacbed_binned > 0])

fig, ax = plt.subplots()
im = ax.imshow(pacbed_binned, interpolation='none', cmap=cmap, vmin=min_nonzero)
cb = fig.colorbar(im, ax=ax)

In [ ]:
dc = d4.crop_bin_sparse_to_dense(frames, frame_size, bin_factor, center, radius_original, frame_dimensions, scan_dimensions, max_frame_size)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=size12)
im1 = ax1.imshow(dc.sum(axis=(0,1)).get())
cb1 = fig.colorbar(im1, ax=ax1)
bf_tot_image = dc.sum(axis=(2,3)).get()
im2 = ax2.imshow(bf_tot_image)
im2.set_clim(np.percentile(bf_tot_image, 1), np.percentile(bf_tot_image, 99))
cb2 = fig.colorbar(im2, ax=ax2)

plt.show()

In [ ]:
# Calculate FFT of 4D data w.r.t. probe position
do_fft_on_gpu = False

t1 = time.perf_counter()

if do_fft_on_gpu:
    dc = dc.astype(cp.float32, copy=False)  # I don't think the copy=False here will actually avoid copying the array
    G = cpfft.fft2(dc, axes=(0, 1), overwrite_x=True)
    del dc
    Gabs = cp.sum(cp.abs(G), (2, 3)).get()
else:
    dc_cpu = cp.asnumpy(dc).astype(np.float32, copy=False)
    del dc
    G_cpu = spfft.fft2(dc_cpu, axes=(0, 1), overwrite_x=True, workers=-1)
    del dc_cpu
    G = cp.asarray(G_cpu)
    Gabs = np.sum(np.abs(G_cpu), (2, 3))
G /= cp.sqrt(np.prod(G.shape[:2]))
sh = np.array(Gabs.shape)

t2 = time.perf_counter()
print(f"FFT along scan coordinate took {t2-t1:.3f}s")

# Mask out some parts of the FFT which are prone to artifacts or noisy background
mask = ~np.array(np.fft.fftshift(d4.sector_mask(sh, sh // 2, 5, (0, 360))))
mask[:,-1] = 0
mask[:,0] = 0
mask[:,1] = 0

mask_half = np.zeros(sh)
m_center = sh // 2
mask_half[:m_center[0]+1, :m_center[1]+1] = 1
if sh[1] % 2 == 0:
    mask_half[m_center[0]+1:, 1:m_center[1]] = 1
else:
    mask_half[m_center[0]+1:, :m_center[1]] = 1
mask_half = np.fft.ifftshift(mask_half)

gg = np.log10(Gabs)
gg[~mask] = gg.mean()

fft_show = np.fft.fftshift(gg)
fig, ax = plt.subplots(1,2,figsize=size12)
ax[0].imshow(np.fft.fftshift(mask))
ax[0].set_title('FFT mask')
ax[1].imshow(fft_show, norm=colors.LogNorm(vmin=np.percentile(fft_show, 0)))
ax[1].set_title('Masked absolute values of G')
plt.show()

In [ ]:
ny, nx, nky, nkx = G.shape

# Spatial spacings and frequencies, assuming everything is square
# scanned positions
rp_min = meta.dr[0] # probe step size
rp_max = rp_min * ny  # this is just FOV
qp_max = 1/rp_min/2 # Nyquist limit
qp_min = 1/rp_max
# cropped/binned CBED
kf_min = meta.alpha_rad/meta.wavelength / radius
rf_max = 1/kf_min # largest "spatial period" in cropped CBED
rf_min = rf_max/nky # real space step size corresponding to how much the CBED was cropped
kf_max = 1/rf_min/2 # Nyquist limit
kf_ssb = meta.alpha_rad/meta.wavelength * 2
rf_ssb = 1/kf_ssb

print('Spatial frequencies, probe positions:')
print(f'Rp: {rp_min:.3f} .. {rp_max:.3f} Å')
print(f'Qp: {qp_max:.3f} .. {rp_min:.3f} Å^-1')
print('Spatial frequencies, cropped/binned CBED:')
print(f'Rf: ({rf_ssb:.3f}) .. {rf_min:.3f} .. {rf_max:.3f} Å')
print(f'Kf: ({kf_ssb:.3f}) .. {kf_max:.3f} .. {kf_min:.3f} Å^-1')

rf_min_arr = np.array([rf_max/nkx, rf_max/nky])
rp_min_arr = np.array(meta.dr)

rf_min_arr *= 1
rp_min_arr *= 1.0
print(rf_min_arr, rp_min_arr)
Kx, Ky = get_qx_qy_1D([nkx, nky], rf_min_arr, dtype=np.float32, fft_shifted=True)
Qx1d, Qy1d = get_qx_qy_1D([nx, ny], rp_min_arr, dtype=np.float32, fft_shifted=False)

n_fit=25
best_angle = meta.rotation_deg
aberrations = cp.zeros((12))

gg = Gabs * mask
gg[gg==0] = gg.mean()

gg_half = gg * mask_half

inds = cp.flip(cp.argsort(gg_half.ravel()))
strongest_inds = np.unravel_index(inds[:n_fit], G.shape[:2])
G_max = G[strongest_inds]

Qy_max1d = Qy1d[strongest_inds[0]]
Qx_max1d = Qx1d[strongest_inds[1]]

dtest, angletest = Q_freq_to_d_spacing(Qx_max1d.get(), Qy_max1d.get())
print('strongest object frequencies')
print(repr_d_spacings(dtest, angletest))
# print(gg[strongest_inds])

Gamma = disk_overlap_function(Qx_max1d, Qy_max1d, Kx, Ky, aberrations, best_angle * np.pi/180, meta.alpha_rad, meta.wavelength)

_, _ = fft_doverlap_figure(np.fft.fftshift(gg), (Qx_max1d.get(), Qy_max1d.get()), gg[strongest_inds], 
    G_max.get() * Gamma.conjugate().get(), G_max.get(), Gamma.get(), figsize=(10,6.5))

In [ ]:
selected_inds = list(range(9))
# order11, order12, order13, order2 = [0, 4], [1, 8], [2, 3], [10, 12, 13]
# selected_inds = order11 + [order2[0]] + order12 + [order2[1]] + order13 + [order2[2]]
G_sel = G_max[selected_inds]
Qx_sel = Qx_max1d[selected_inds]
Qy_sel = Qy_max1d[selected_inds]

In [ ]:
# allocate output arrays for SSB
Psi_Qp = cp.zeros((ny, nx), dtype=np.complex64)
Psi_Qp_left_sb = cp.zeros((ny, nx), dtype=np.complex64)
Psi_Qp_right_sb = cp.zeros((ny, nx), dtype=np.complex64)

eps = 1e-3

In [ ]:
# SSB with some defocus value

aberrations[0] = 0.0
print(f'defocus: {aberrations[0]}')

start = time.perf_counter()

single_sideband_reconstruction(
    G,
    Qx1d, Qy1d, Kx, Ky,
    aberrations,
    best_angle * np.pi/180,
    meta.alpha_rad,
    Psi_Qp, Psi_Qp_left_sb, Psi_Qp_right_sb,
    eps,
    meta.wavelength,
)

Psi_Rp_left_sb = cpfft.ifft2(Psi_Qp_left_sb, norm="ortho")
Psi_Rp_right_sb = cpfft.ifft2(Psi_Qp_right_sb, norm="ortho")
Psi_Rp = cpfft.ifft2(Psi_Qp, norm="ortho")

ssb_defocal = Psi_Rp.get()
ssb_defocal_right = Psi_Rp_right_sb.get()
ssb_defocal_left = Psi_Rp_left_sb.get()

print(f"SSB took {time.perf_counter() - start}")

my_ssb_img = np.angle(ssb_defocal_left)
my_fft = np.fft.fftshift(np.fft.fft2(my_ssb_img))
f_crop = my_fft.shape[0]//6
fft_show = np.abs(my_fft[f_crop:-f_crop, f_crop:-f_crop])

fig, ax = plt.subplots(1,2,figsize=size12)
im1 = ax[0].imshow(my_ssb_img, cmap=plt.cm.get_cmap('bone'))
ax[0].set_title(f'Scan SSB ptychography')
ax[0].set_xticks([])
ax[0].set_yticks([])
# fig.colorbar(im1, ax=ax[0])
ax[0].add_artist(ScaleBar(meta.dr[0]/10,'nm'))

ax[1].imshow(fft_show, norm=colors.LogNorm(vmin=np.percentile(fft_show, 30)))
plt.show()

In [ ]:
from ipywidgets import AppLayout

C_gui = np.zeros((12,))
C = cp.zeros((12,))
C_exp = np.array([2,1, 3,3, 6,5,5])  # what power of 10 to scale the aberration coefficents by

# for scan 45 (to actually use the values in the GUI, might need to force a refresh of the figure by adjusting a parameter)
# C_gui, C, C_exp = np.array([0.1, 2.0, 88, 0, 0, 0, 0, 0, 0, 0, -9.0, -48]), cp.zeros(12), np.array([2, 1, 3, 3, 5, 5, 4])

# Using a numpy array becauase this somehow allows it to be modified from the UI
dp_angle_arr = np.array([best_angle/180 * np.pi])

# If the UI elements don't fit well, try changing the size of your window or the figure DPI
# plt.rcParams['figure.dpi'] = 90

AppLayout(center=manual_aberration_ui(G, G_sel, C, C_gui, dp_angle_arr, 
        Psi_Qp, Psi_Qp_left_sb, Psi_Qp_right_sb, 
        Qx1d, Qy1d, Qx_sel, Qy_sel, Kx, Ky, eps,
        aberrations, meta.alpha_rad, meta.wavelength, meta.dr[0],
        C_exp=C_exp))

In [ ]:
print(C_gui)
print(C_exp)
print(C)
print(f'{dp_angle_arr[0] * 180 / np.pi:.1f}')

In [ ]:
# Do reconstruction with previously selected aberrations

start = time.perf_counter()
eps = 1e-3
single_sideband_reconstruction( # want this function
    G,
    Qx1d,
    Qy1d,
    Kx,
    Ky,
    C,
    dp_angle_arr[0],
    meta.alpha_rad,
    Psi_Qp,
    Psi_Qp_left_sb,
    Psi_Qp_right_sb,
    eps,
    meta.wavelength,
)

Psi_Rp_left_sb = cpfft.ifft2(Psi_Qp_left_sb, norm="ortho")
Psi_Rp_right_sb = cpfft.ifft2(Psi_Qp_right_sb, norm="ortho")
Psi_Rp = cpfft.ifft2(Psi_Qp, norm="ortho")

ssb_defocal = Psi_Rp.get()
ssb_defocal_right = Psi_Rp_right_sb.get()
ssb_defocal_left = Psi_Rp_left_sb.get()

fig, axs = plt.subplots(1,2, figsize=size12)
im1 = axs[0].imshow(np.angle(ssb_defocal_right), cmap= plt.cm.get_cmap('bone'))
axs[0].set_title(f'SSB ptychography')
axs[0].set_xticks([])
axs[0].set_yticks([])
fig.colorbar(im1, ax=axs[0])
axs[0].add_artist(ScaleBar(meta.dr[0]/10,'nm'))

my_ssb_image = np.angle(ssb_defocal_right)
my_fft = np.fft.fftshift(np.fft.fft2(my_ssb_image))
fft_show = np.abs(my_fft)
im2 = axs[1].imshow(fft_show, norm=colors.LogNorm(vmin=np.percentile(fft_show, 50)))

plt.tight_layout()
plt.show()

In [ ]:
tifffile.imwrite(results_path /f'scan{scan_number}_ssb_ptycho_best_right.tif',np.angle(ssb_defocal_right).astype('float32'), 
        imagej=True, resolution=(1./(meta.dr[0]/10), 1./(meta.dr[1]/10)), 
        metadata={'spacing': 1 / 10, 'unit': 'nm', 'axes': 'YX'})